# Matchmaking!

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('profiles_revised.csv.zip')
df.sample(3)

In [ ]:
df.shape

## Simple Idea: Find Commonalities

In [ ]:
df['body_type'].value_counts()

In [ ]:
df['drinks'].value_counts()

### Putting People Together

Suppose we want to find a match for this person:

In [ ]:
match_me = df.iloc[0]
match_me

In [ ]:
matches = df[(df['body_type'] == match_me['body_type'])
            & (df['drinks'] == match_me['drinks'])]
matches.shape

### Assuring Sexual Compatibility

In [ ]:
full_matches = matches[(matches['sex'] == 'f') & (matches['orientation'] == 'straight')]
full_matches.sample(3)

In [ ]:
full_matches.shape

## More Sophisticated Matching

### One-Hot Encoding

In [ ]:
vars_that_matter = ['diet', 'drinks', 'drugs', 'smokes']

In [ ]:
filtered = df[vars_that_matter]

In [ ]:
filtered.isna().sum()

In [ ]:
si = SimpleImputer(strategy='most_frequent')

si.fit(filtered)

In [ ]:
imputed = pd.DataFrame(si.transform(filtered),
                       columns=filtered.columns,
                       index=filtered.index)
imputed.sample(3)

In [ ]:
ohe = OneHotEncoder(sparse=False)

ohe.fit(imputed)

In [ ]:
encoded = pd.DataFrame(ohe.transform(imputed),
                      columns=ohe.get_feature_names(),
                      index=imputed.index)
encoded.sample(3)

Here is our to-be-matched person again, now encoded:

In [ ]:
match_me_encoded = encoded.iloc[0].values.reshape(1, 32).flatten()

### Cosine Similarity

In [ ]:
numerators = np.array([match_me_encoded.dot(other) for other in encoded.iloc[1:].values])

In [ ]:
denominators = np.array([np.sqrt(sum(match_me_encoded**2)) *\
                         np.sqrt(sum(other**2)) for other in encoded.iloc[1:].values])

In [ ]:
sim_scores = numerators / denominators

In [ ]:
sim_scores_full = [1]
sim_scores_full.extend(sim_scores)

In [ ]:
encoded['similarity'] = sim_scores_full

In [ ]:
encoded[encoded['similarity'] == 1]

In [ ]:
metrics.pairwise.cosine_similarity(encoded[:10])

### Putting Variables on a Scale